In [1]:
import numpy as np
import trimesh
import geopandas as gpd
import pandas as pd
from shapely import geometry
from skimage import measure

from rbf.interpolate import KNearestRBFInterpolant as KNRBFInterpolant
import matplotlib.pyplot as plt

In [2]:
from boreholes_helper import Boreholes, BoreholeModel

In [3]:
from scipy import signal
from scipy.interpolate import RBFInterpolator

In [4]:
# PANDAS CONFIG
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = None

# PLT CONFIG
plt.rcParams['figure.figsize'] = [15, 7]

In [5]:
# a = Boreholes()
a = BoreholeModel()

In [6]:
a.setCollar(
    path='./BAKAN_COLLAR.CSV',
    x='UTM_EAST',
    y='UTM_NORTH',
    z='RL',
    depth='TDEPTH',
    keys=['BHID'],
    id='BHID',
)

a.readCollar()
a.applyCollarConfigMap()
a.createCollarPointGeometry()
a.updateCollarCrs(32651)

# create metric geometry
df = a.getDataFrame('collar')
df = Boreholes.changeCrs(df, srid=32651)
df['geometryMetric'] = df['geometry']
df = Boreholes.changeCrs(df, srid=4326)
a.setDataFrame('collar', df)

a.setCollar(srid=4326)
a.createCollarLineSegment(key='geometryMetric')
# a.viewCollarLine().show()

/media/lelegoyang/OS/DevTeam/jupyter_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [7]:
a.setSurvey(
    path='./BAKAN_SURVEY.CSV',
    depth='AT',
    dip='DM_DIP',
    azimuth='UTM_BRG',
    keys=['BHID'],
    id='BHID',
)
a.readSurvey()
a.applySurveyConfigMap()

In [8]:
a.setInterval(
    path='./BAKAN_ASSAY.CSV',
    z0='FROM',
    z1='TO',
    keys=['BHID'],
    id='BHID',
    x='UTM_EAST',
    y='UTM_NORTH',
    z='RL',
)
a.readInterval()
a.applyIntervalConfigMap()

In [9]:
areas = {
    'a': { # why the-f they keep raising singular matrix error in this area?
        'xmin': 112.30,
        'xmax': 112.305,
        'ymin': 0.565,
        'ymax': 0.58,
    },
    'b': {
        'xmin': 112.3157,
        'xmax': 112.32,
        'ymin': 0.575,
        'ymax': 0.6,
    },
    'c': {
        'xmin': 112.30,
        'xmax': 112.312,
        'ymin': 0.54,
        'ymax': 0.565,
    },
    'd': { # two borehole
        'xmin': 112.30,
        'xmax': 112.302,
        'ymin': 0.564,
        'ymax': 0.565,
    },
}

In [13]:
area_ = 'all'
bbox = areas.get(area_)
a.setCollar(bbox=bbox)
collar = a.getDataFrame('collar')
collar['geometry'].explore()